# Image Processing QSTP

### Dataset - MNIST Fashion
### Model - 2 Layer CNN along with maxpool downscalling + 2 Connected Layers
### Optimizer - SGD + Nesterov Momentum

## Step 0. Necessary Imports

In [15]:
#Necessary imports
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms 
import torchvision.datasets as dsets
from torch.autograd import Variable
import os
import gzip
import urllib.request
import os.path
import torch.utils.data as data
import codecs
from PIL import Image

## Step 1. Create and Load Dataset

### 1.1 Dataset Class of MNIST Fashion
P.S. Please put files in the "./data/".

In [16]:
class FashionMNISTDataset(data.Dataset):
    '''Fashion MNIST Dataset'''
    
    urls = 	[
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz'
            ]

    file_name =	[
                    'train-images-idx3-ubyte',
                    'train-labels-idx1-ubyte',
                    't10k-images-idx3-ubyte',
                    't10k-labels-idx1-ubyte'
                ]
    
    training_file = 'training.pt'
    test_file = 'test.pt'
    
    def __init__(self, source , train=True, transform=None):
        
        self.source = source
        
        if os.path.exists(source) == False:
            self.download(self.source)
        
        if self.check_processed(train=train, source=source) is not True:
            self.process_files(train=train,source=source);
        
        if train is True:
            self.X, self.Y = torch.load(
                os.path.join(source, self.training_file))
        else:
            self.X, self.Y = torch.load(
                os.path.join(source, self.test_file))
        
        self.transform = transform;
        
    def __len__(self):
        return len(self.X);
    
    def check_processed(self, train, source):
        return os.path.exists(os.path.join(source, self.training_file)) and \
            os.path.exists(os.path.join(source, self.test_file))
    
    def __getitem__(self, idx):
        item = self.X[idx];
        label = self.Y[idx];
        
        item = Image.fromarray(item.numpy(), mode='L')
        
        if self.transform:
            item = self.transform(item);
        
        return (item, label);
    
    def download(self, source):
        
        try:
			      os.makedirs(self.source)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
              raise
        pass
        
        for file_index in range(len(self.file_name)):
            print("Downloading:",self.urls[file_index])
            urllib.request.urlretrieve(self.urls[file_index],(self.file_name[file_index]+'.gz'))
            f = gzip.open(self.file_name[file_index]+'.gz', 'rb')
            with open(source+self.file_name[file_index],'wb+') as w:
                for line in f.readlines():
                    w.write(line)
            f.close()
            os.remove(self.file_name[file_index]+".gz")
    
    @classmethod
    def process_files(self, train, source):
        
        if train is True:
            imgs='train-images-idx3-ubyte'
            lbls='train-labels-idx1-ubyte'
            n=60000
        else:
            imgs='t10k-images-idx3-ubyte'
            lbls='t10k-labels-idx1-ubyte'
            n=10000
        
        f = open(str(source+imgs), "rb")        
        l = open(str(source+lbls), "rb")
        
        to_set = (
            read_image_file(f),
            read_label_file(l)
        )
        
        if train is True:
            fh = open(str(source+self.training_file), 'wb+')
        else:
            fh = open(str(source+self.test_file), 'wb+')
        
        torch.save(to_set, fh)
        
        fh.close()
        
        f.close()
        l.close()

        
def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)


def read_label_file(f):
    data = f.read()
    length = get_int(data[4:8])
    parsed = np.frombuffer(data, dtype=np.uint8, offset=8)
    return torch.from_numpy(parsed).view(length).long()


def read_image_file(f):
    data = f.read()
    length = get_int(data[4:8])
    num_rows = get_int(data[8:12])
    num_cols = get_int(data[12:16])
    parsed = np.frombuffer(data, dtype=np.uint8, offset=16)
    return torch.from_numpy(parsed).view(length, num_rows, num_cols)

### 1.2 Load Dataset

In [17]:
train_dataset = FashionMNISTDataset(source='./data/',
                                    train=True,
                                    transform=transforms.ToTensor())
test_dataset = FashionMNISTDataset(source='./data/',
                                   train=False,
                                   transform=transforms.ToTensor())

Downloading: http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
Downloading: http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
Downloading: http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz
Downloading: http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz


## 2. Make Dataset iterable

In [18]:
batch_size = 100
no_iters = 12000
no_epochs = no_iters / ( len(train_dataset) / batch_size )
no_epochs = int(no_epochs)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## 3. Create Model Class

In [19]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        #Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.norm1 = nn.BatchNorm2d(32)
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.norm2 = nn.BatchNorm2d(64)
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        #FNN 1
        self.fc1 = nn.Linear(64*7*7, 64*7*7)
        self.relu3 = nn.ReLU()
        
        #Linear
        self.fc2 = nn.Linear(64*7*7, 10)
        
    def forward(self,x):
        #Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.norm1(out)
        
        #Max pool 1
        out = self.maxpool1(out)
        
        #Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.norm2(out)
        
        #Max pool 2
        out = self.maxpool2(out)
        
        #Resize
        #original size: (100, 64, 7, 7)
        #out.size(0): 100
        #New out size: (100, 64*7*7)
        out = out.view(out.size(0), -1)
        
        #FNN 1
        out = self.fc1(out)
        out = self.relu3(out)
        
        #Linear
        out = self.fc2(out)
        
        return out

## 4. Instantiate Model Class

In [20]:
if torch.cuda.is_available():
    model =CNNModel().cuda()
else:
    model = CNNModel()

## 5. Instantiate Loss Class

In [21]:
criterion = nn.CrossEntropyLoss()

## 6. Instantiate Optimizer Class

In [22]:
learning_rate = 0.01
moment = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = moment, nesterov = True)

## 7. Train Model

In [23]:
iter = 1
for epoch in range(no_epochs):
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    for i,(images,labels) in enumerate(train_loader):
        #Variables
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
            
        #Clear Gradient
        optimizer.zero_grad()
        
        #Outpus
        outputs = model(images)
        
        #Calculate loss: softmax -> cross entropy loss
        loss = criterion(outputs, labels)
        
        #Generate gradients w.r.t. parameters
        loss.backward()
        
        #Update Parameters
        optimizer.step()
        
        iter += 1
        if iter %2000 ==0:
            #Calculate Accuracy
            correct=0
            total=0
            train_accuracy = 0
            test_accuracy = 0
            
            #Train Accuracy
            for images,labels in train_loader:
                if torch.cuda.is_available():
                    images = images.cuda()
                #outputs
                outputs = model(images)
                
                #Get predictions from maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
                total += labels.size(0)
                
                #Total Correct Labels
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                    
            train_accuracy = 100 * float(correct) / total
            correct=0
            total=0
            
            #Test Accuracy
            for images,labels in test_loader:
                if torch.cuda.is_available():
                    images = images.cuda()
                #outputs
                outputs = model(images)
                
                #Get predictions from maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
                total += labels.size(0)
                
                #Total Correct Labels
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                    
            test_accuracy = 100 * float(correct) / total
            
            print('Iteration: {}, Loss: {}, Train Accuracy: {}, Test Accuracy: {}'.format(iter, loss.item(), train_accuracy, test_accuracy))

Iteration: 2000, Loss: 0.17604094743728638, Train Accuracy: 94.92, Test Accuracy: 91.16
Iteration: 4000, Loss: 0.04010731354355812, Train Accuracy: 98.23833333333333, Test Accuracy: 91.95
Iteration: 6000, Loss: 0.035265859216451645, Train Accuracy: 99.66333333333333, Test Accuracy: 92.17
Iteration: 8000, Loss: 0.0002535819949116558, Train Accuracy: 99.98333333333333, Test Accuracy: 92.37
Iteration: 10000, Loss: 0.00018728256691247225, Train Accuracy: 100.0, Test Accuracy: 92.56
Iteration: 12000, Loss: 0.0002989387430716306, Train Accuracy: 100.0, Test Accuracy: 92.57
